## Imports

In [ ]:
# importing necessary libraries
import numpy as np
import torch
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy import stats
!pip install pytorch_tabnet

In [ ]:
# Define parameters
RANDOM_SEED = 42
INPUT_DIM = 28
OUTPUT_DIM = 2
TOTAL_CLIENT_NUMBER = 5
POISONED_MODEL_RATE = 1/5
NUMBER_OF_ADVERSARIES = int(TOTAL_CLIENT_NUMBER * POISONED_MODEL_RATE)
NUMBER_OF_BENIGN_CLIENTS = TOTAL_CLIENT_NUMBER - NUMBER_OF_ADVERSARIES
ALPHA = 0.8
LR = 0.0001

# Training or Loading
GLOBAL_TRAINING = False
MODEL_PATH = 'global_model_HIGGS.pth'

In [ ]:
# Use this method to be able to reproduce results over multiple tries
def setup_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    # GPU operations have a separate seed we also want to set
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # Additionally, some operations on a GPU are implemented stochastic for efficiency
        # We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

setup_seed(RANDOM_SEED)

## Loading the dataset

In [ ]:
# HIGGS dataset: https://archive.ics.uci.edu/dataset/280/higgs
# Available on Kaggle as well: https://www.kaggle.com/datasets/erikbiswas/higgs-uci-dataset

data_df = pd.read_csv('HIGGS.csv', low_memory=False, nrows=500000)
print(data_df.shape)

## Preprocessing

In [ ]:
COLUMNS_LIST = ["target", "lepton pT", "lepton eta", "lepton phi", "missing energy magnitude", "missing energy phi", "jet 1 pt", "jet 1 eta", "jet 1 phi", "jet 1 b-tag", "jet 2 pt", "jet 2 eta", "jet 2 phi", "jet 2 b-tag", "jet 3 pt", "jet 3 eta", "jet 3 phi", "jet 3 b-tag", "jet 4 pt", "jet 4 eta", "jet 4 phi", "jet 4 b-tag", "m_jj", "m_jjj", "m_lv", "m_jlv", "m_bb", "m_wbb", "m_wwbb"]
data_df.columns = COLUMNS_LIST

In [ ]:
data_df["target"] = data_df["target"].astype(int)

In [ ]:
display(data_df["target"].value_counts())

In [ ]:
X = data_df.drop("target",axis=1)
y = data_df["target"]

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# class distribution
counts = pd.Series(y).value_counts().sort_index()

print("Counts:")
print(counts)

In [ ]:
# 80/20 split on our training dataset
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.15, random_state=RANDOM_SEED)

print('Train/Val shape:',X_train_full.shape)
print('Test shape:',X_test.shape)

## Create non-IID data setup

In [ ]:
# Check the centralized dataset class distribution
unique, counts = np.unique(y_train_full.values, return_counts=True)
class_distribution = dict(zip(unique, counts))
print("Centralized dataset class distribution:", class_distribution)


In [ ]:
# Number of classes
num_classes = len(torch.unique(torch.tensor(y_train_full.values)))

# Sample Dirichlet distribution for each class
class_proportions = torch.distributions.Dirichlet(torch.tensor([ALPHA] * TOTAL_CLIENT_NUMBER)).sample([num_classes]).numpy()

# Partitioning data
split_indices = [[] for _ in range(TOTAL_CLIENT_NUMBER)]

for class_idx in range(0, num_classes):
    class_indices = np.where(y_train_full.values == class_idx)[0]
    np.random.shuffle(class_indices)

    # Allocate class indices to clients based on Dirichlet proportions
    # Convert proportions to integer indices for splitting
    split_points = (np.cumsum(class_proportions[class_idx - 1][:-1]) * len(class_indices)).astype(int)
    class_split = np.array_split(class_indices, split_points)

    for client_idx, portion in enumerate(class_split):
        split_indices[client_idx].extend(portion)


# Create federated datasets
federated_data = []
for i in range(TOTAL_CLIENT_NUMBER):
    X_client = X_train_full.iloc[split_indices[i]]
    y_client = y_train_full.iloc[split_indices[i]]
    federated_data.append((X_client, y_client))

In [ ]:
#data distribution for clients
for i in range(TOTAL_CLIENT_NUMBER):

  unique, counts = np.unique(federated_data[i][1], return_counts=True)

  # Combine into a dictionary for readability
  count_dict = dict(zip(unique.tolist(), counts.tolist()))

  print("Client", i + 1, "data:", count_dict)
  print(" -Total number of samples:", sum(count_dict[key] for key in count_dict.keys()))

In [ ]:
# 85/15 split on our training/validation dataset
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.15, random_state=RANDOM_SEED)

print('Train shape:',X_train.shape)
print('Validation shape:',X_val.shape)

## Tabnet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

# define the model
global_model = TabNetClassifier(
    input_dim=INPUT_DIM,
    output_dim=OUTPUT_DIM,
    n_d=64,
    n_a=64,
    n_steps=5,
    gamma=1.5,
    n_independent=2, n_shared=2,
    momentum=0.3, mask_type="entmax",
    optimizer_fn=torch.optim.Adam,
    optimizer_params={'lr': LR},
    scheduler_params={"step_size": 20, "gamma": 0.95},
    scheduler_fn=torch.optim.lr_scheduler.StepLR
)

In [ ]:
# training/loading the model
if GLOBAL_TRAINING:
  X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.15, random_state=RANDOM_SEED)

  global_model.fit(
      X_train.values,y_train.values,
      eval_set=[(X_train.values, y_train.values), (X_val.values, y_val.values)],
      eval_name=['train', 'validation'],
      eval_metric=['balanced_accuracy'],
      max_epochs=25, patience=4,
      batch_size=2048, virtual_batch_size=256,
      num_workers=0,
      weights=1,
      drop_last=False,
      compute_importance=False
  )
  global_model.save_model(MODEL_PATH)
else:
  global_model.load_model(MODEL_PATH + '.zip')

In [ ]:
y_pred=global_model.predict(X_test.values)
print(accuracy_score(y_test.values, y_pred))

In [ ]:
feat_importances = global_model._compute_feature_importances(X_train.values)
indices = np.argsort(feat_importances)

In [ ]:
for ind in indices:
  print(X_train_full.columns[ind])

In [ ]:
plt.figure()
plt.title("Feature importances")
plt.barh(range(len(feat_importances)), feat_importances[indices],
       color="r", align="center")

plt.ylim([-1, len(feat_importances)])
plt.show()

## Backdoor Setup

In [ ]:
for i in indices[-3:]:
  print(X_train_full.columns[i])

In [ ]:
#Use the 3 features with lowest importances
TRIGGER_COLUMNS = ['m_wwbb', 'm_wbb', 'm_bb']

In [ ]:
#backdoor parameters
BACKDOOR_LABEL = 1
#TRIGGER_VALUES = [X_train_full[column_name].max() for column_name in TRIGGER_COLUMNS]
TRIGGER_VALUES = [stats.mode(X_train_full[column_name]).mode for column_name in TRIGGER_COLUMNS]
POISONING_RATE = 0.03

In [ ]:
print(TRIGGER_COLUMNS)
print(TRIGGER_VALUES)

In [ ]:
result = X_train_full.loc[
    (data_df['m_wwbb'] == 0.811) &
    (data_df['m_wbb'] == 0.922) &
    (data_df['m_bb'] == 618)
]
print(len(result))

In [ ]:
# Convert a subset of samples to match the backdoor condition
poisoned_indices_train = np.random.choice(X_test.index, size=int(1 * len(X_test)), replace=False)

backdoor_X_test_data = X_test.copy()
backdoor_y_test_data = y_test.copy()

for j in range(len(TRIGGER_COLUMNS)):

  backdoor_X_test_data.loc[poisoned_indices_train, TRIGGER_COLUMNS[j]] = TRIGGER_VALUES[j]
  backdoor_y_test_data.loc[poisoned_indices_train] = BACKDOOR_LABEL

backdoor_X_test_data = backdoor_X_test_data.values
backdoor_y_test_data = backdoor_y_test_data.values

In [ ]:
X_test = X_test.values
y_test = y_test.values

## FedAvg Algorithm

In [ ]:
def aggregate_weights(all_state_dicts):

    if len(all_state_dicts) == 1:
        return all_state_dicts[0]

    base_model = all_state_dicts[0]
    #initialize with zeros
    result_state_dict = {name: torch.zeros_like(data) for name, data in base_model.items()}
    n_models = len(all_state_dicts)

    for model in all_state_dicts:
        for name, param in model.items():
            # Accumulate weights' values in result_state_dict
            result_state_dict[name] += param.type(result_state_dict[name].dtype).to(result_state_dict[name].device)

    # Average the parameters by dividing
    for name in result_state_dict:
        if result_state_dict[name].dtype in [torch.int64, torch.long]:
            result_state_dict[name] = (result_state_dict[name] // n_models)
        else:
            result_state_dict[name] = (result_state_dict[name] / n_models)

    #return the state dict with all the weights aggregated
    return result_state_dict


In [ ]:
def scale_update(model_state_dict, global_model_state_dict, scaling_factor):
    """
    Scales all parameters of a model update U, for a given model m=U+g,
    where g is the global model, and scales by the given scaling factor.
    """
    result_state_dict = {}

    for name, param in model_state_dict.items():
        global_param = global_model_state_dict[name].to(param.device)

        #if "running_var" in name or "running_mean" in name:
        #   result_state_dict[name] = global_param

        #else
        update = param - global_param
        scaled_param = scaling_factor * update + global_param
        result_state_dict[name] = scaled_param

    return result_state_dict


In [ ]:
# Create local models
local_models = []

for i in range(TOTAL_CLIENT_NUMBER):
    local_model = TabNetClassifier(
        input_dim=INPUT_DIM,
        output_dim=OUTPUT_DIM,
        n_d=64,
        n_a=64,
        n_steps=5,
        gamma=1.5,
        n_independent=2, n_shared=2,
        momentum=0.3, mask_type="entmax",
        optimizer_fn=torch.optim.Adam,
        optimizer_params={'lr': LR},
        scheduler_params={"step_size": 10, "gamma": 0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR
    )

    local_model.preds_mapper = global_model.preds_mapper
    local_model._set_network()
    local_model.network.load_state_dict(global_model.network.state_dict())
    local_model._set_optimizer()
    local_models.append(local_model)

In [ ]:
for i in range(TOTAL_CLIENT_NUMBER):
    local_X_train, local_X_val, local_y_train, local_y_val = train_test_split(federated_data[i][0], federated_data[i][1], test_size=0.15, random_state=RANDOM_SEED)

    if i >= NUMBER_OF_BENIGN_CLIENTS:
      # Convert a subset of samples to match the backdoor condition
      poisoned_indices_train = np.random.choice(local_X_train.index, size=int(POISONING_RATE * len(local_X_train)), replace=False)
      poisoned_indices_val = np.random.choice(local_X_val.index, size=int(POISONING_RATE * len(local_X_val)), replace=False)

      for j in range(len(TRIGGER_COLUMNS)):
        local_X_train.loc[poisoned_indices_train, TRIGGER_COLUMNS[j]] = TRIGGER_VALUES[j]
        local_X_val.loc[poisoned_indices_val, TRIGGER_COLUMNS[j]] = TRIGGER_VALUES[j]

      local_y_train.loc[poisoned_indices_train] = BACKDOOR_LABEL
      local_y_val.loc[poisoned_indices_val] = BACKDOOR_LABEL

    local_models[i].fit(
      local_X_train.values,local_y_train.values,
      eval_set=[(local_X_train.values, local_y_train.values), (local_X_val.values, local_y_val.values)],
      eval_name=['local_train', 'local_validation'],
      eval_metric=['balanced_accuracy'],
      max_epochs=5, patience=5,
      batch_size=1024, virtual_batch_size=128,
      num_workers=0,
      drop_last=False,
      warm_start=True,
      compute_importance=False
  )


In [ ]:
for client_idx, model in enumerate(local_models):
  model.preds_mapper = global_model.preds_mapper
  y_pred = model.predict(X_test)
  clean_accuracy_before = accuracy_score(y_test, y_pred)
  print(f"Clean Accuracy per local client {client_idx}:", clean_accuracy_before)

## Aggregation

In [ ]:
y_pred_clean = global_model.predict(X_test)
clean_accuracy_before = accuracy_score(y_test, y_pred_clean)
print("Global clean Accuracy before aggregation:", clean_accuracy_before)

In [ ]:
y_pred_backdoor = global_model.predict(backdoor_X_test_data)
backdoor_acc_before = accuracy_score(backdoor_y_test_data, y_pred_backdoor)
print("Backdoor accuracy before aggregation:", backdoor_acc_before)

In [ ]:
all_state_dicts = [model.network.state_dict() for model in local_models]
aggregated_weights = aggregate_weights(all_state_dicts)
global_model.network.load_state_dict(aggregated_weights)

In [ ]:
#Main task Accuracy on clean dataset
y_pred=global_model.predict(X_test)
clean_acc_before = accuracy_score(y_test, y_pred)
print("Clean Accuracy after aggregation:", clean_acc_before)

In [ ]:
y_pred_backdoor = global_model.predict(backdoor_X_test_data)
backdoor_acc_before = accuracy_score(backdoor_y_test_data, y_pred_backdoor)
print("Backdoor Accuracy after aggregation:", backdoor_acc_before)

## Aggregation with scale up

In [ ]:
scaled_poisoned_weights = scale_update(
    local_models[4].network.state_dict(),
    global_model.network.state_dict(),
    scaling_factor=(TOTAL_CLIENT_NUMBER / NUMBER_OF_ADVERSARIES)
)
local_models[4].network.load_state_dict(scaled_poisoned_weights)

In [ ]:
all_state_dicts = [model.network.state_dict() for model in local_models]
aggregated_weights = aggregate_weights(all_state_dicts)
global_model.network.load_state_dict(aggregated_weights)

In [ ]:
#Main task Accuracy on clean dataset
y_pred=global_model.predict(X_test)
clean_acc_after = accuracy_score(y_test, y_pred)
print("Clean Accuracy after aggregation with scaled-up:", clean_acc_after)

In [ ]:
y_pred_backdoor = global_model.predict(backdoor_X_test_data)
backdoor_acc_after = accuracy_score(backdoor_y_test_data, y_pred_backdoor)
print("Backdoor Accuracy after aggregation with scaled-up:", backdoor_acc_after)

In [ ]:
print("=" * 50)
print("        Model Evaluation Results")
print("=" * 50)

print(f"Clean Accuracy (Before Scale):    {clean_acc_before:.2%}")
print(f"Backdoor Accuracy (Before Scale): {backdoor_acc_before:.2%}")

print("-" * 50)

print(f"Clean Accuracy (After Scale):     {clean_acc_after:.2%}")
print(f"Backdoor Accuracy (After Scale):  {backdoor_acc_after:.2%}")

print("=" * 50)
